In [539]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from collections import Counter
import random
import time
import numpy as np
import math

In [2]:
url = 'https://classic.warcraftlogs.com/zone/reports?zone=1001'

response = requests.get(url)
page = response.text

soup = BeautifulSoup(page,"lxml")



In [6]:
table=soup.find('table')

In [7]:
rows = [row for row in table.find_all('tr')]  # tr tag is for rows

In [702]:
wow_logs = {}

for i in range(1,20):
    url = 'https://classic.warcraftlogs.com/zone/reports?zone=1001&page='+str(i)
    response = requests.get(url)
    page = response.text

    soup = BeautifulSoup(page,"lxml")
    table=soup.find('table')
    rows = [row for row in table.find_all('tr')]
    
    for row in rows:
        #gets report link
        link = row.find_all('td')[0].find('a')['href']
    
        #name of guild
        guild = row.find_all('td')[2].text
    
        wow_logs[link] = [link,guild]


len(wow_logs)

1900

In [742]:
# Start up remote window
from selenium import webdriver
driver = webdriver.Firefox(executable_path=r'C:/Users/atag3/Desktop/NYC DSA Python EDA/Untitled Folder/geckodriver.exe')
driver.get('https://classic.warcraftlogs.com/reports/b2tKdk1xmj7pw43z')

# we have to click through the first message accepting cookies. this should only nee to be done once.
butt = driver.find_elements_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/button[3]')
butt[0].click()

In [341]:
def check_exists_by_xpath(xpath):
    try:
        webdriver.driver.find_elements_by_class_name(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
#new_logs={}
counter_place = 1410
#now we want to go through each link
for links in list(wow_logs.keys())[1410:len(wow_logs.keys())]:
    # Navigate to the fight summary page
    link = 'https://classic.warcraftlogs.com'+links
    driver.get(link)
    
    # Look for boss fights recorded in the log and return the 3rd boss fight. It should always be onyxia
    p_element = driver.find_elements_by_class_name("report-overview-boss-caption")
    
    if len(p_element)==0:
        pass
    else:

        #click the onyxia fight to give us the fight report
        p_element[2].click()
        time.sleep(.5+2*random.random())
    
        #at this point we have made it to the fight summary page we want.

        # First I want to make sure we are looking at Onyxia
        boss_icon = driver.find_elements_by_id("filter-fight-boss-icon")
        is_ony = 'Onyxia' if re.search('1084-icon.jpg',boss_icon[0].get_attribute('src')) else 'Not Onyxia'

        #look at fight time and status
        kill_wipe= driver.find_elements_by_id("filter-fight-details-text")

        #is this a wipe or kill?
        is_kill = kill_wipe[0].text.split(" (")[0]

        #how long did it take? format in MM:SS
        time_f = kill_wipe[0].text.split('(')[1].split(')')[0]

        # look at character details
        player_info = driver.find_elements_by_class_name("character-details-contents")
        chars = player_info[0].text.split('\n')
        classes = [i for i in chars if '(' in i] 
        # make a dict of class (spec) counts
        classes = Counter(classes)

        # Average Item Level
        tanks_end=chars.index('DPS')
        dps_end=chars.index('Healers')
        heals_end=len(chars)

        #Average Ilvl of tanks
        ilvls = [re.findall("\d+", i) for i in chars[0:tanks_end] if 'Item Level' in i]
        flattened_ilvl = [int(val) for sublist in ilvls for val in sublist]
        tank_ilvls = np.mean(flattened_ilvl)
        
        #Average Ilvl of DPS
        ilvls = [re.findall("\d+", i) for i in chars[tanks_end:dps_end] if 'Item Level' in i]
        flattened_ilvl = [int(val) for sublist in ilvls for val in sublist]
        dps_ilvls = np.mean(flattened_ilvl)
        
        #Average Ilvl of Healers
        ilvls = [re.findall("\d+", i) for i in chars[dps_end:heals_end] if 'Item Level' in i]
        flattened_ilvl = [int(val) for sublist in ilvls for val in sublist]
        heals_ilvls = np.mean(flattened_ilvl)
        
        # World Buff Info
        buff_info = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[6]/div[3]/div[1]/div[7]/div[3]/div[2]/div[7]/div[2]/div[*]/div/table/tbody/tr[*]/td[3]/a[*]/img')
        w_buff = len(buff_info)
        
        new_logs[links] = {'boss':is_ony,
                           'kill':is_kill,
                           'fight_time':time_f,
                           'tank_avg_ilvl': tank_ilvls,
                           'dps_avg_ilvl': dps_ilvls,
                           'tank_avg_ilvl': heals_ilvls,
                           'world_buffs': w_buff
                          }
        new_logs[links].update(dict(zip(list(classes.keys()),list(classes.values()))))
        new_logs[links]
        
        
        counter_place += 1

    

In [741]:
counter_place

1410

In [700]:
df = pd.DataFrame.from_dict(new_logs, orient='index')
df

,boss,kill,fight_time,tank_avg_ilvl,dps_avg_ilvl,world_buffs,Druid (Guardian),Druid (Balance),Hunter (Marksmanship),Mage (Frost),...,Rogue (Assassination),Shaman (Enhancement),Warlock (Destruction),Warrior (Fury),Druid (Restoration),Priest (Holy),Shaman (Restoration),Warrior (Protection),Druid (Feral),Paladin (Holy)
/reports/bDLBVNcJAHxdfzMG,Onyxia,Last Pull - Kill,4:49,70.750000,68.727273,2,2.0,1.0,4,2.0,...,4.0,1.0,3.0,4,2.0,3,3.0,NaN,NaN,NaN
/reports/hk34VT6AJRcyPBnD,Onyxia,Last Pull - Kill,16:16,71.000000,68.600000,1,NaN,NaN,1,1.0,...,NaN,NaN,NaN,2,1.0,1,NaN,1.0,1.0,1.0
/reports/aGW4rJwgKcRCYqTM,Onyxia,Kill,3:22,69.875000,69.153846,0,1.0,1.0,1,8.0,...,5.0,1.0,3.0,6,1.0,3,4.0,NaN,NaN,NaN
/reports/TDc4zKdQjW1CxbLM,Onyxia,Last Pull - Kill,4:56,70.600000,70.538462,36,1.0,NaN,1,NaN,...,4.0,NaN,1.0,5,NaN,2,3.0,1.0,NaN,NaN
/reports/PYRgjh7aJzN3L9Kp,Onyxia,Kill,5:05,69.000000,69.117647,7,NaN,NaN,3,3.0,...,4.0,NaN,3.0,3,2.0,2,NaN,1.0,NaN,2.0
/reports/BT8NLnr24GFVkDHZ,Onyxia,Last Pull - Kill,1:55,72.555556,71.680000,70,NaN,NaN,2,NaN,...,7.0,NaN,2.0,6,NaN,5,4.0,2.0,NaN,NaN


In [ ]:
df.to_pickle('wowlogs_data.pkl')

In [456]:
from flatten_dict import flatten
flat_logs = flatten(new_logs)
df = pd.DataFrame.from_dict(flat_logs, orient='columns').T

ValueError: If using all scalar values, you must pass an index

dict_keys(['Druid (Guardian)', 'Druid (Balance)', 'Hunter (Marksmanship)', 'Mage (Frost)', 'Priest (Shadow)', 'Rogue (Assassination)', 'Shaman (Enhancement)', 'Warlock (Destruction)', 'Warrior (Fury)', 'Druid (Restoration)', 'Priest (Holy)', 'Shaman (Restoration)'])

In [311]:
logs_df = pd.DataFrame(wow_logs).T  #transpose
logs_df.columns = ['link', 'title', 'guild', 
                    'time']

logs_df.head()

ValueError: arrays must all be same length

In [306]:
# we need to look into next step of looking at logs and making sure they are just onyxia
link = 'https://classic.warcraftlogs.com/'+'/reports/b2tKdk1xmj7pw43z'
response = requests.get(link)
page = response.text

In [127]:
soup = BeautifulSoup(page,"lxml")
soup.find(class_='fight_duration')

soup.find_all('div', attrs={'class':'fight-duration'})

#<img onerror="this.src = defaultBossImage()" class="fight-selection-boss-icon" src="https://assets.rpglogs.com/img/warcraft/bosses/1084-icon.jpg">
#<span class="report-overview-boss-kill"><span style="white-space:nowrap; line-height:17px" class="kill"> Kill <span class="fight-duration">(9:07)</span> <span class="fight-time" style="color:#ddd;font-style:italic;font-size:11px; margin-left:4px">6:59 AM </span></span></span>

[]

In [129]:
r = requests.get(link, timeout=(3.05, 27))

In [131]:
page = r.text

In [133]:
soup = BeautifulSoup(page,"lxml")
soup.find(class_='fight_duration')

soup

#<img onerror="this.src = defaultBossImage()" class="fight-selection-boss-icon" src="https://assets.rpglogs.com/img/warcraft/bosses/1084-icon.jpg">
#<span class="report-overview-boss-kill"><span style="white-space:nowrap; line-height:17px" class="kill"> Kill <span class="fight-duration">(9:07)</span> <span class="fight-time" style="color:#ddd;font-style:italic;font-size:11px; margin-left:4px">6:59 AM </span></span></span>

<!DOCTYPE html>
<html lang="en">
<head>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
(function() {
  var host = 'warcraftlogs.com';
  var element = document.createElement('script');
  var firstScript = document.getElementsByTagName('script')[0];
  var url = 'https://quantcast.mgr.consensu.org'
    .concat('/choice/', 'KvP_65z5XueLh', '/', host, '/choice.js')
  var uspTries = 0;
  var uspTriesLimit = 3;
  element.async = true;
  element.type = 'text/javascript';
  element.src = url;

  firstScript.parentNode.insertBefore(element, firstScript);

  function makeStub() {
    var TCF_LOCATOR_NAME = '__tcfapiLocator';
    var queue = [];
    var win = window;
    var cmpFrame;

    function addFrame() {
      var doc = win.document;
      var otherCMP = !!(win.frames[TCF_LOCATOR_NAME]);

      if (!otherCMP) {
        if (doc.body) {
          var iframe = doc.createElement('iframe');

          iframe.style.cssText = 'display

In [566]:
# Start up remote window
from selenium import webdriver
driver = webdriver.Firefox(executable_path=r'C:/Users/atag3/Desktop/NYC DSA Python EDA/Untitled Folder/geckodriver.exe')
driver.get('https://classic.warcraftlogs.com/reports/2GvdYWAzCFbxcKnD#fight=last')

In [232]:
# we have to click through the first message accepting cookies
butt = driver.find_elements_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/button[3]')
butt[0].click()


In [233]:
# Look for boss fights recorded in the log and return the 3rd boss fight. It should always be onyxia
p_element = driver.find_elements_by_class_name("report-overview-boss-caption")

#click the onyxia fight to give us the fight report
p_element[2].click()

In [226]:
#we should now be on the log kill report. I am not totally sure how it works with wipes and multiple attemps

'Onyxia Normal\nKill (8:03) 4:37 PM'

In [308]:
#at this point we have made it to the fight summary page we want.
new_dict = {}
# First I want to make sure we are looking at Onyxia
boss_icon = driver.find_elements_by_id("filter-fight-boss-icon")
is_ony = 'Onyxia' if re.search('1084-icon.jpg',boss_icon[0].get_attribute('src')) else 'Not Onyxia'

#look at fight time and status
kill_wipe= driver.find_elements_by_id("filter-fight-details-text")

#is this a wipe or kill?
is_kill = kill_wipe[0].text.split(" (")[0]

#how long did it take? format in MM:SS
time = kill_wipe[0].text.split('(')[1].split(')')[0]

# look at character details
player_info = driver.find_elements_by_class_name("character-details-contents")
chars = player_info[0].text.split('\n')
classes = [i for i in chars if '(' in i] 
# make a dict of class (spec) counts
classes = Counter(classes)

In [295]:
##############################################################################################
# Saving link as a concrete example of wipe data https://classic.warcraftlogs.com/reports/3ArGvzf6kJC9jaBm#fight=1
driver.get('https://classic.warcraftlogs.com/reports/rD2fPLFxYNTyCwV7#fight=last')

kill_wipe= driver.find_elements_by_id("filter-fight-details-text")
is_kill = kill_wipe[0].text.split(" (")[0]
#kill time value
time = kill_wipe[0].text.split('(')[1].split(')')[0]

                              
############################################################################################

In [316]:
print(classes)

Counter({'Hunter (Marksmanship)': 3, 'Rogue (Assassination)': 3, 'Priest (Holy)': 3, 'Warrior (Protection)': 2, 'Mage (Frost)': 2, 'Warlock (Destruction)': 2, 'Warrior (Fury)': 2, 'Paladin (Holy)': 2, 'Druid (Feral)': 1, 'Mage (Fire)': 1, 'Druid (Restoration)': 1})


In [305]:
Counter(classes)

Counter({'Warrior (Protection)': 2,
         'Druid (Feral)': 1,
         'Hunter (Marksmanship)': 3,
         'Mage (Frost)': 2,
         'Mage (Fire)': 1,
         'Rogue (Assassination)': 3,
         'Warlock (Destruction)': 2,
         'Warrior (Fury)': 2,
         'Druid (Restoration)': 1,
         'Paladin (Holy)': 2,
         'Priest (Holy)': 3})